In [1]:
import logging
import string
import os
import pickle
from pprint import pprint
import re 
import numpy as np
import pandas as pd
from IPython.display import clear_output
from more_itertools import chunked

# NLTK
import nltk
from nltk.corpus import stopwords

# Gensim
from gensim import corpora, models, similarities
from gensim.models.coherencemodel import CoherenceModel
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
import gensim

# SpaCy 
import spacy
import en_core_web_sm
nlp = en_core_web_sm.load()

# Plotting tools
!pip install pyLDAvis
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis
pyLDAvis.enable_notebook()

import pyLDAvis
import matplotlib.pyplot as plt
%matplotlib inline

# Logging if you want it
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)
# Suppressing it should you want to
logging.getLogger().setLevel(logging.CRITICAL)

# Suppressing warnings
import warnings
warnings.simplefilter("ignore", DeprecationWarning)

#SKLearn
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import LatentDirichletAllocation
from sklearn.feature_extraction import text

In [2]:
path=os.listdir(r"C:\Users\Li\Desktop\新加坡（txt）\split")
datalist=[]

for i in path:
    domain= r"C:\\Users\\Li\\Desktop\\新加坡（txt）\\split\\"+i
    #print(domain)
    with open(domain,"r",encoding="utf-8") as f:
        data=f.read()
        datalist.append(data)
len(datalist)

260

In [11]:
data2 = [re.sub(r'[^a-zA-Z\.:/]',' ',txt) for txt in datalist]
               
data1 = [txt.replace('\n', ' ') for txt in data2]

no_url = [re.sub('((www\.[^\s]+)|(https?://[^\s]+))', '', txt) for txt in data1]

no_line= [txt.replace('/', ' ').replace(':','').replace('.',' ') for txt in no_url]

data = [re.sub(r'\s+', ' ', txt) for txt in no_line]

data[8]

'A New Zealand judge sentenced white supremacist Brenton Tarrant to life in prison without parole on Thursday Aug for killing Muslim worshippers in the country s deadliest shooting saying the sentence was not enough punishment for the wicked crimes It was the first time a court in New Zealand had sentenced a person to prison for the rest of their life Prime Minister Jacinda Ardern said Tarrant deserved a lifetime of complete and utter silence Survivor Ms Hina Amir said justice was served Read also New Zealand mosque gunman s road to extremism S PORE TEAM STUDYING HEART DAMAGE IN COVID PATIENTS A team of doctors and researchers in Singapore has started a cardiac research programme to better understand why some patients who are hit hard by Covid also suffer heart damage and how to prevent this from happening The National University Heart Centre Singapore NUHCS team aims to recruit Covid patients as mounting evidence overseas suggests some patients continue to suffer long term complicatio

In [12]:
def tokenizer(texts):
    for text in texts:
        yield(gensim.utils.simple_preprocess(text, deacc=True))

token_list=list(tokenizer(data))

In [13]:
# Build bigram and trigram models
# min_count = min amount of tokens/bigrams in corpus. Threshold = score-based; higher = fewer phrases.
bigram = gensim.models.Phrases(token_list, min_count=5, threshold=100)
trigram = gensim.models.Phrases(bigram[token_list], threshold=100)  

# `Phraser` must be built from an initial `Phrases` instance. 
# It then works faster while using much less memory. See https://radimrehurek.com/gensim/models/phrases.html
bigram_mod = gensim.models.phrases.Phraser(bigram)
trigram_mod = gensim.models.phrases.Phraser(trigram)

stop = set(stopwords.words('english')+['ASIANREVIEW','com','cn','Straits'])


def remove_stopwords(texts):
    return [[word for word in doc if word not in stop] for doc in texts]
      
def make_bigrams(texts):
    return [bigram_mod[doc] for doc in texts]

def make_trigrams(texts):
    return [trigram_mod[bigram_mod[doc]] for doc in texts]

def lemmatization(texts, allowed_postags=['NOUN']):
    """https://spacy.io/api/annotation"""
    texts_out = []
    for doc in texts:
        joined = nlp(" ".join(doc)) 
     
        texts_out.append([token.lemma_ for token in joined if token.pos_ in allowed_postags])
    return texts_out

In [14]:
tokens_no_stops = remove_stopwords(token_list)

print(tokens_no_stops[0][1])

crisis


In [15]:
# Form trigrams
trigrams = make_trigrams(tokens_no_stops)

# Do lemmatization 
lemmas = lemmatization(trigrams)

In [16]:
# Create Dictionary 
dictionary = corpora.Dictionary(lemmas)

# Create Corpus, i.e. Document-Term Matrix
corpus = [dictionary.doc2bow(text) for text in lemmas]

In [17]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=dictionary,
                                           num_topics=8, 
                                           random_state=100,
                                           # eval_every = 20, # this is evaluation, perplexity
                                           update_every=1,
                                           chunksize=500,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [18]:
lda_viz = gensimvis.prepare(lda_model, corpus, dictionary)
lda_viz

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
5      0.043809 -0.030669       1        1  48.902845
1      0.079739 -0.007558       2        1  12.295917
0     -0.038484 -0.048110       3        1  10.523281
2      0.054484 -0.013379       4        1  10.334411
3      0.026313 -0.036946       5        1   6.841148
4     -0.139639 -0.063770       6        1   4.763795
7      0.040954  0.058772       7        1   3.907199
6     -0.067175  0.141660       8        1   2.431404, topic_info=           Term         Freq        Total Category  logprob  loglift
11        china  2191.000000  2191.000000  Default  30.0000  30.0000
611   singapore   437.000000   437.000000  Default  29.0000  29.0000
36        india   259.000000   259.000000  Default  28.0000  28.0000
99      country   662.000000   662.000000  Default  27.0000  27.0000
87         year   316.000000   316.000000  Default  26.0000  26.0000
..          ...          ...          ...      ...      ...      ...
59        power     3.653986   219.686680   Topic8  -5.7341  -0.3797
32   government     3.647463   248.871558   Topic8  -5.7359  -0.5062
444    province     3.275437    56.335443   Topic8  -5.8434   0.8718
139     project     3.369431   225.890688   Topic8  -5.8151  -0.4886
118       group     3.140140    81.474747   Topic8  -5.8856   0.4607

[610 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
3344      1  0.263382          abe
3344      3  0.175588          abe
3344      4  0.614557          abe
4371      5  0.822942  accommodate
892       4  0.609942          acm
...     ...       ...          ...
546       1  0.418687         zone
546       2  0.055825         zone
546       3  0.418687         zone
546       4  0.055825         zone
546       6  0.055825         zone

[1403 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[6, 2, 1, 3, 4, 5, 8, 7])